In [24]:
import torch
import torch.nn as nn
import cv2
import os
import pandas as pd
from torchvision import transforms
from sklearn.model_selection import train_test_split

In [25]:
symbols_keys = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J',
    20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O', 25: 'P', 26: 'Q', 27: 'R', 28: 'S', 29: 'T',
    30: 'U', 31: 'V', 32: 'W', 33: 'X', 34: 'Y', 35: 'Z'
}

In [26]:
class CharsDataset(torch.utils.data.Dataset):
  def __init__(self, path, transform = None):
    super().__init__()
    self.data = pd.read_csv(path)
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    row = self.data.iloc[idx]
    label = row[0]
    pixels = row[1:].values.astype('float32')/255.0
    image = pixels.reshape(28,28)

    if self.transform:
      image = self.transform(image)
    return image, label


In [27]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(10),
])

In [47]:
dataset = CharsDataset('C:/Users/vipvo/Jupyter_Programs/typedCSV без строчных.csv', transform = transform)
train_data, val_data = train_test_split(dataset, test_size = 0.2)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = 16, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size = 1)

C:\Users\vipvo\AppData\Local\Temp\ipykernel_6832\1769068122.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[0]


In [48]:
class TextModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1, stride = 1)
        self.conv1 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 3, padding = 1, stride = 1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size = 3, padding = 1, stride = 1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size = 3, padding = 1, stride = 1)
        
        self.act = nn.LeakyReLU()
        self.pooling = nn.MaxPool2d(2)
        self.linear0 = nn.Linear(128, 256)
        self.linear1 = nn.Linear(256, 36)
        # self.linear2 = nn.Linear(128, 36)

        self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))

    def forward(self, x):
        out = self.conv0(x)
        out = self.act(out)
        out = self.pooling(out)


        out = self.conv1(out)
        out = self.act(out)
        out = self.pooling(out)
        

        out = self.conv2(out)
        out = self.act(out)
        out = self.pooling(out)

        # out = self.conv3(out)
        # out = self.act(out)
        # out = self.pooling(out)

        # out = self.conv4(out)
        # out = self.act(out)
        
        out = self.adaptivepool(out)
        

        out = out.view(out.size(0), -1)
        
        out = self.linear0(out)
        out = self.act(out)

        out = self.linear1(out)
        # out = self.act(out)

        # out = self.linear2(out)
        return out
        

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
def train_model(model, dataloader, num_epochs=30, num_classes=36):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for image, label in dataloader:
            image = image.to(device)
            label = nn.functional.one_hot(label, num_classes).float()
            label = label.to(device)
            optimizer.zero_grad()
            predict = model(image)
            loss = loss_fn(predict, label)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}')

In [51]:
train_model(model, train_loader)

Epoch 1/30, Loss: 1.3850
Epoch 2/30, Loss: 0.3393
Epoch 3/30, Loss: 0.2362
Epoch 4/30, Loss: 0.1835
Epoch 5/30, Loss: 0.1467
Epoch 6/30, Loss: 0.1182
Epoch 7/30, Loss: 0.1022
Epoch 8/30, Loss: 0.0878
Epoch 9/30, Loss: 0.0781
Epoch 10/30, Loss: 0.0684
Epoch 11/30, Loss: 0.0633
Epoch 12/30, Loss: 0.0565
Epoch 13/30, Loss: 0.0584
Epoch 14/30, Loss: 0.0500
Epoch 15/30, Loss: 0.0552
Epoch 16/30, Loss: 0.0454
Epoch 17/30, Loss: 0.0491
Epoch 18/30, Loss: 0.0451
Epoch 19/30, Loss: 0.0437
Epoch 20/30, Loss: 0.0404
Epoch 21/30, Loss: 0.0428
Epoch 22/30, Loss: 0.0361
Epoch 23/30, Loss: 0.0395
Epoch 24/30, Loss: 0.0363
Epoch 25/30, Loss: 0.0382
Epoch 26/30, Loss: 0.0340
Epoch 27/30, Loss: 0.0345
Epoch 28/30, Loss: 0.0318
Epoch 29/30, Loss: 0.0302
Epoch 30/30, Loss: 0.0308


In [52]:
scriped_model = torch.jit.script(model)
scriped_model.save('scriped_Number_to_Text_ONLY_BIG.pt')

In [53]:
def val_model(model, dataloader, num_classes=36):
    model.eval()
    total_loss = 0
    right_pred = 0
    false_pred = 0
    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            labels = nn.functional.one_hot(label, num_classes).float()
            label = label.to(device)
            labels = labels.to(device)
            predict = model(image)
            loss = loss_fn(predict, labels)
            total_loss += loss.item()

            if label.item() == torch.argmax(predict):
                right_pred += 1
            else:
                false_pred += 1
        print(f'Loss: {total_loss/len(dataloader):.4f}\nRight:\
        {right_pred}/{len(dataloader)}; False: {false_pred}/{len(dataloader)}')

In [54]:
val_model(model, val_loader)

Loss: 0.1440
Right:        7055/7316; False: 261/7316


In [36]:
#torch.save(model, 'NumberToText.pt')


In [37]:
def try_model(model_path, file_path, CSV_idx=None, label=None, img_size=(28, 28)):
    model = torch.load(model_path)
    name, existion = os.path.splitext(file_path)
    if existion == '.csv':    
        file = pd.read_csv(file_path)              
        row = file.iloc[CSV_idx]
        image = row[1:]
        image = image.values
        label = row[0] 
        
    else:
        print('Для изображений пока не добавил функционал')

    image = image.astype('float32')/255.0
    image = image.reshape(28,28)
    resized_image = cv2.resize(image, img_size, interpolation=cv2.INTER_LINEAR)
    cv2.imshow('Window', resized_image)
    image = torch.tensor(image).unsqueeze(0).unsqueeze(0).to(device)
    
    result = model(image)
    res_symb = torch.argmax(result).item()
    print(f'Prediction: {res_symb} ({symbols_keys[res_symb]}); Label: {label}')
    cv2.waitKey(0) 
    cv2.destroyAllWindows()


In [38]:
try_model('scriped_Number_to_Text_ONLY_BIG.pt', 'typedCSV без строчных.csv', 12866, img_size=(250, 250))

C:\Users\vipvo\AppData\Local\Temp\ipykernel_6832\3681034531.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
D:\Programs\Python\Lib\site-pa

Prediction: 12 (C); Label: 12


In [39]:
#СДЕЛАТЬ ИНТЕГРАЦИЮ С YOLO